In [1]:
#MySQL 연결
import pymysql
from sqlalchemy import create_engine, text #연결 관리용으로만 가볍게
from pathlib import Path
from pprint import pprint

#데이터 처리/시각화
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#외부 API 호출하기
import requests
from requests.exceptions import Timeout, ConnectionError, RequestException

#.env 파일 활용하기
from dotenv import load_dotenv
import os
import time

#env 파일 읽기
load_dotenv()

#API 요청 정보
url = 'https://bigdata.kepco.co.kr/openapi/v1/powerUsage/houseAve.do'
api_key = os.getenv('ELECTRIC_API_KEY')

#db에서 level 1인 지역 코드를 가져왔다. 앞의 두자리를 잘라서
#사용할 예정입니다.
region = [
"1100000000","2600000000","2700000000","2800000000","2900000000","3000000000","3100000000","3600000000","4100000000","4300000000","4400000000","4600000000","4700000000","4800000000","5000000000","5100000000","5200000000"
]

#SQLAlchemy의 engine을 미리 만들어 놓습니다.
DB_URL = f"mysql+pymysql://{os.getenv('DB_USERNAME')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
engine = create_engine(DB_URL)

In [2]:
#engine 연결을 확인합니다.
with engine.connect() as conn:
    result = conn.execute(text("SELECT 1"))
    print(result.fetchone())  # (1,)

(1,)


In [13]:
#단건의 간단한 호출 테스트 할 수 있습니다.
url = 'https://bigdata.kepco.co.kr/openapi/v1/powerUsage/houseAve.do'
api_key = os.getenv('ELECTRIC_API_KEY')

params = {
    'year' : '2025',
    'month' : '11',
    'metroCd' : '11',
    'apiKey' : os.getenv('ELECTRIC_API_KEY')
}

response = requests.get(url, params=params)

if response.status_code == 200:
    df = pd.DataFrame(response.json()['data'])
    print(len(df))
else:
    print(f'Erorr: {response.status_code}')


25


In [11]:
def convert_input(data):
    return {
        'sd_code' : int(data.get('sd_code')),
        'year' : int(data.get('year')),
        'month' : int(data.get('month')),
        'sd_name' : data.get('metro'),
        'sgg_name' : data.get('city'),
        'house_cnt' : int(data.get('houseCnt')),
        'power_usage' : float(data.get('powerUsage')),
        'bill' : int(data.get('bill'))
    }

def save_batch(data_list):
    """배치 데이터 DB 저장 - Raw SQL"""
    if not data_list:
        return
    
    # 실제 컬럼명에 맞게 수정 필요
    query = text("""
        INSERT INTO household_power 
        (sd_code, year, month, sd_name, sgg_name, house_cnt, power_usage, bill)
        VALUES (:sd_code, :year, :month, :sd_name, :sgg_name, :house_cnt, :power_usage, :bill)
    """)
    
    with engine.connect() as conn:
        conn.execute(query, data_list)
        conn.commit()

def is_already_collected(year, month, region_code):
    """이미 수집했는지 확인"""
    query = text("""
        SELECT COUNT(*) as cnt 
        FROM household_power 
        WHERE year=:year AND month=:month AND sd_code=:sd_code
    """)
    
    with engine.connect() as conn:
        result = conn.execute(query, {'year': year, 'month': month, 'sd_code': region_code})
        return result.fetchone()[0] > 0

fail_list = []
# 메인 로직
for year in range(2013, 2026):
    for month in range(1, 13):
        if (year == 2013 and month < 5) or (year == 2025 and month > 11):
            continue
        
        for r in region:
            region_code = r[:2]

            #이미 수집된 날짜, 지역의 경우 API 요청을 스킵합니다.
            if is_already_collected(year, month, region_code):
                # print(f"{year}-{month:02d} {region_code}: 이미 수집됨")
                continue

            #API의 요청 조건을 담는 params
            params = {
                'year' : year,
                'month' : f'{month:02d}',
                'metroCd' : region_code,
                'apiKey' : os.getenv('ELECTRIC_API_KEY')
            }
            
            try:
                response = requests.get(url, params=params, timeout=5)
                response.raise_for_status()
                result = response.json()

                #row_data에는 제외되어 있는 지역코드를 추가해줍니다.
                if 'data' in result and result['data']:
                    for data in result['data'] :
                        data['sd_code'] = region_code

                    #convert_input함수를 통해 dict 리스트의 형태를 테이블과 맞도록 변형해주었습니다.
                    convert_data = [convert_input(d) for d in result['data']]
                    # print(f"{year}-{month:02d} {region_code}: {len(result['data'])}건 입력됨")
                    save_batch(convert_data)  # Raw SQL로 저장
                        
            except Exception as e:
                print(f"✗ {year}-{month:02d} {region_code}: {type(e).__name__}")
                fail_list.append([year, f'{month:02d}', region_code])
            
            time.sleep(5)